<a href="https://colab.research.google.com/github/sjgrider256/Digital_Forest/blob/main/Species_Classification%3A_Train_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install DeepForest
!pip install deepforest

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
fixed_raster_path = "/content/drive/My Drive/Remote Sensing Projects/GeoTiffs/Fixed_Clear-Brooks-Drive-3-13-2025.tif"


Downsample Ortho to Xcm/pixel

In [ ]:

import rasterio
from rasterio.enums import Resampling

with rasterio.open(fixed_raster_path) as src:
    #scale the ortho
    scale_factor = 10 / 1.5
    new_width = int(src.width / scale_factor)
    new_height = int(src.height / scale_factor)
    #smooth over the image taking the average of groups of 4 pixels at a time during downsampling
    data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.bilinear
    )
    #Use .transform to scale geospatial data (match change in geospatial data to pixel change using scale factor)
    new_transform = src.transform * src.transform.scale(
        #x-scale
        src.width / new_width,
        #y-scale
        src.height / new_height
    )
  #update the new meta data
    profile = src.profile
    profile.update({
        'height': new_height,
        'width': new_width,
        'transform': new_transform
    })
  #write this new metadata to a the file.
    with rasterio.open("downsampled.tif", "w", **profile) as dst:
        dst.write(data)



In [ ]:
from deepforest import main

# Initialize DeepForest
model = main.deepforest()
model.load_model(model_name="weecology/deepforest-tree", revision="main")

# Use the new downsampled raster
downsampled_path = "downsampled.tif"  # or full path if needed

# Predict on the downsampled image
predicted_results = model.predict_tile(
    downsampled_path,
    patch_size=1000,
    patch_overlap=0.25
)

# Print results
print("\n📌 Predictions Before Conversion:")
print(predicted_results.head())


Reading config file: /usr/local/lib/python3.11/dist-packages/deepforest/data/deepforest_config.yml


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 148MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Reading config file: /usr/local/lib/python3.11/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

1888 predictions in overlapping windows, applying non-max supression
1014 predictions kept after non-max suppression

📌 Predictions Before Conversion:
     xmin    ymin    xmax    ymax label     score       image_path  \
0  3115.0  1500.0  3202.0  1579.0  Tree  0.795434  downsampled.tif   
1  1696.0  1811.0  1773.0  1893.0  Tree  0.713757  downsampled.tif   
2  3416.0  1245.0  3492.0  1322.0  Tree  0.693220  downsampled.tif   
3  1715.0  1416.0  1798.0  1495.0  Tree  0.688943  downsampled.tif   
4  3875.0  1044.0  3925.0  1091.0  Tree  0.683127  downsampled.tif   

                                            geometry  
0  POLYGON ((3202 1500, 3202 1579, 3115 1579, 311...  
1  POLYGON ((1773 1811, 1773 1893, 1696 1893, 169...  
2  POLYGON ((3492 1245, 3492 1322, 3416 1322, 341...  
3  POLYGON ((1798 1416, 1798 1495, 1715 1495, 171...  
4  POLYGON ((3925 1044, 3925 1091, 3875 1091, 387...  


Save as CSV

In [ ]:
predictions_output_path = "CSV_Predictions13.csv"
predicted_results.to_csv(predictions_output_path, index=False)
print(f"\n✅ Predictions saved to: {predictions_output_path}")



✅ Predictions saved to: CSV_Predictions13.csv


Downlaod CSV (move to correct folder on local drive).

In [ ]:
from google.colab import files
files.download("CSV_Predictions13.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

convert x & y min and max coordinates to geographical coordinatees

In [ ]:
from deepforest import utilities
import os

# ✅ Paths inside Colab
predictions_path = "CSV_Predictions13.csv"
raster_path = "downsampled.tif"  # Not an absolute path, just the filename

# ✅ Read Predictions
print("\n📌 Reading Predictions CSV...")
df = utilities.read_file(predictions_path)

# ✅ Convert to Geographic Coordinates
print("\n📌 Converting to Geographic Coordinates...")
geo_df = utilities.image_to_geo_coordinates(df, root_dir=".")

# ✅ Print sample to verify
print("\n✅ Sample Converted Predictions:")
print(geo_df.head())



📌 Reading Predictions CSV...

📌 Converting to Geographic Coordinates...

✅ Sample Converted Predictions:
     xmin    ymin    xmax    ymax label     score       image_path  \
0  3115.0  1500.0  3202.0  1579.0  Tree  0.795434  downsampled.tif   
1  1696.0  1811.0  1773.0  1893.0  Tree  0.713757  downsampled.tif   
2  3416.0  1245.0  3492.0  1322.0  Tree  0.693220  downsampled.tif   
3  1715.0  1416.0  1798.0  1495.0  Tree  0.688943  downsampled.tif   
4  3875.0  1044.0  3925.0  1091.0  Tree  0.683127  downsampled.tif   

                                            geometry  
0  POLYGON ((648185.105 3899813.647, 648185.105 3...  
1  POLYGON ((648048.975 3899784.021, 648048.975 3...  
2  POLYGON ((648212.731 3899837.938, 648212.731 3...  
3  POLYGON ((648051.356 3899821.649, 648051.356 3...  
4  POLYGON ((648253.98 3899857.086, 648253.98 389...  


Convert predicitons to shapefile

In [ ]:
import geopandas as gpd

# ✅ Define output path (in Colab's working directory)
shapefile_output_path = "Tree_Predictions_Shapefile.shp"

# ✅ Save as shapefile
geo_df.to_file(shapefile_output_path)

print(f"\n✅ Predictions saved as a shapefile: {shapefile_output_path}")



✅ Predictions saved as a shapefile: Tree_Predictions_Shapefile.shp


Downlaod shapefile and save to correct folder on local drive


In [ ]:
import zipfile
import os

# ✅ Zip all parts of the shapefile
with zipfile.ZipFile("Tree_Predictions_Shapefile.zip", "w") as zipf:
    for ext in [".shp", ".shx", ".dbf", ".prj", ".cpg"]:
        filename = f"Tree_Predictions_Shapefile{ext}"
        if os.path.exists(filename):
            zipf.write(filename)

# ✅ Download the zip
from google.colab import files
files.download("Tree_Predictions_Shapefile.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>